In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from bokeh.io import export_svgs, export_png
from holoviews import opts, dim
from holoviews.operation import histogram
hv.extension('bokeh')
from bokeh.resources import INLINE

import importlib
import processing_parameters
import datetime

import functions_bondjango as bd
import functions_plotting as fp
import pandas as pd
import numpy as np
import pprint as pp
import paths


In [ ]:
# set up the figure config
importlib.reload(fp)
importlib.reload(processing_parameters)
# define the target saving path
save_path = os.path.join(paths.figures_path, 'Behavior_basic')

# define the printing mode
save_mode = True
# define the target document
target_document = 'paper'
# set up the figure theme
fp.set_theme()
# load the label dict
label_dict = processing_parameters.label_dictionary
variable_list = processing_parameters.variable_list

In [ ]:
# load the data

importlib.reload(processing_parameters)

# get the search list
search_list = processing_parameters.search_list
# allocate memory for the data
full_df = []

# allocate a list for all paths (need to preload to get the dates)
all_paths = []
# all_results = []
# for all the search strings
for search_string in search_list:

    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_all = [el for el in data_all if 'behconsolidate' in el['slug']]
    data_path = [el['analysis_path'] for el in data_all if '_behconsolidate' in el['slug']]
#     data_result = [el['result'] for el in data_all if '_preproc' in el['slug']]
    all_paths.append(data_path)
#     all_results.append(data_result)
# # get the dates present
# data_dates = np.unique([os.path.basename(el)[:10] for el in np.concatenate(all_paths)])
# print(f'Dates present: {data_dates}')

# allocate memory for the resulting dataframe

# for all the list items
for idx0, data_path in enumerate(all_paths):

    # for all the files
    for idx1, files in enumerate(data_path):
        # load the dataframe into the output list
        full_df.append(pd.read_hdf(files, 'data'))
    
print(len(full_df))

# convert to a single dataframe
full_df = pd.concat(full_df, axis=0)
print(full_df.shape)
        

In [ ]:
# plot the performance over days

print(full_df.columns[:50])
# group by the result
performance = full_df.groupby(['day', 'mouse'], as_index=False)[['day', 'result']].mean()

# allocate memory for the delta time df
performance_list = []
sub_df = []
# for all animals
for idx, (animal, df) in enumerate(performance.groupby(['mouse'])):
    # exclude the habituation rows
    df = df.iloc[df.loc[:, 'result'].to_numpy() >= 0, :]
    # reset the index of the sub dataframe
    df.reset_index(drop=True, inplace=True)

    # get the delta time
    delta_time = [(el-df['day'][0]).days for el in df['day']]
    df.loc[:, 'day'] = delta_time
    
    sub_df.append(df)
    
#     df = df.iloc[both, :]
    # build the label
    label = 'M'+str(idx)
    # plot and save
    perf_plot = hv.Curve(df[['day', 'result']], label=label)
    perf_plot.opts(width=600, height=600, ylabel='Performance (a.u.)', xlabel='Time (days)')
#     perf_plot.opts(color='black', alpha=0.2)
#     perf_plot.opts(ylim=(1, 6))
#     perf_plot = fp.format_figure(perf_plot, width=600, height=400)
    performance_list.append(perf_plot)

# sub_mean = pd.concat(sub_df, axis=0).groupby(['day'], as_index=False).mean()
# sub_sem = pd.concat(sub_df, axis=0).groupby(['day'], as_index=False).sem()
# average_plot = hv.Curve(sub_mean[['day', 'result']])
# sem_plot = hv.Spread((sub_mean['day'], sub_mean['result'], sub_sem['result']))
# performance_list.append(average_plot)
# performance_list.append(sem_plot)

overlay = hv.Overlay(performance_list)
overlay.opts(legend_position='right')
overlay

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'performance')) + '.png')
# save the figure
fig = fp.save_figure(overlay, save_name, fig_width=12, dpi=1200, fontsize=target_document, target='both')

In [ ]:
# Plot hunt duration over days

# group by the result
duration = full_df.groupby(['day', 'mouse'], as_index=False)[['day', 'duration']].mean()

# allocate memory for the delta time df
duration_list = []
# for all animals
for idx, (animal, df) in enumerate(duration.groupby(['mouse'])):
    # exclude the habituation rows
    df = df.iloc[df.loc[:, 'duration'].to_numpy() >= 0, :]
    # reset the index of the sub dataframe
    df.reset_index(drop=True, inplace=True)

    # get the delta time
    delta_time = [(el-df['day'][0]).days for el in df['day']]
    df.loc[:, 'day'] = delta_time
    
#     df = df.iloc[both, :]
    # build the label
    label = 'M'+str(idx)
    # plot and save
    dur_plot = hv.Curve(df[['day', 'duration']], label=label)
    dur_plot.opts(width=600, height=600, ylabel='Duration (s)', xlabel='Time (days)')

#     perf_plot.opts(ylim=(1, 6))
#     perf_plot = fp.format_figure(perf_plot, width=600, height=400)
    duration_list.append(dur_plot)

overlay = hv.Overlay(duration_list)
overlay.opts(legend_position='right')
overlay

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'duration')) + '.png')
# save the figure
fig = fp.save_figure(overlay, save_name, fig_width=12, dpi=1200, fontsize=target_document, target='both')

In [ ]:
# plot the approach number over time
# group by the result
approachN = full_df.groupby(['day', 'mouse'], as_index=False)[['day', 'approach_number']].mean()

# allocate memory for the delta time df
approachN_list = []
# for all animals
for idx, (animal, df) in enumerate(approachN.groupby(['mouse'])):
    # exclude the habituation rows
    df = df.iloc[df.loc[:, 'approach_number'].to_numpy() >= 0, :]
    # reset the index of the sub dataframe
    df.reset_index(drop=True, inplace=True)

    # get the delta time
    delta_time = [(el-df['day'][0]).days for el in df['day']]
    df.loc[:, 'day'] = delta_time
    
#     df = df.iloc[both, :]
    # build the label
    label = 'M'+str(idx)
    # plot and save
    appN_plot = hv.Curve(df[['day', 'approach_number']], label=label)
#     perf_plot.opts(ylim=(1, 6))
#     perf_plot = fp.format_figure(perf_plot, width=600, height=400)
    approachN_list.append(appN_plot)

hv.Overlay(approachN_list)

In [ ]:
# plot the latency to first approach

# group by the result
latencyapp = full_df.groupby(['day', 'mouse'], as_index=False)[['day', 'latency_approach']].mean()

# allocate memory for the delta time df
latency_approach_list = []
# for all animals
for idx, (animal, df) in enumerate(latencyapp.groupby(['mouse'])):
    # exclude the habituation rows
    df = df.iloc[df.loc[:, 'latency_approach'].to_numpy() >= 0, :]
    # reset the index of the sub dataframe
    df.reset_index(drop=True, inplace=True)

    # get the delta time
    delta_time = [(el-df['day'][0]).days for el in df['day']]
    df.loc[:, 'day'] = delta_time
    
#     df = df.iloc[both, :]
    # build the label
    label = 'M'+str(idx)
    # plot and save
    latapp_plot = hv.Curve(df[['day', 'latency_approach']], label=label)
    latapp_plot.opts(width=600, height=600, ylabel='Latency to hunt (s)', xlabel='Time (days)')

#     perf_plot.opts(ylim=(1, 6))
#     perf_plot = fp.format_figure(perf_plot, width=600, height=400)
    latency_approach_list.append(latapp_plot)

overlay = hv.Overlay(latency_approach_list)
overlay.opts(legend_position='right')
overlay

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'latency')) + '.png')
# save the figure
fig = fp.save_figure(overlay, save_name, fig_width=12, dpi=1200, fontsize=target_document, target='both')